In [120]:
import numpy as np
import math 
import matplotlib.pyplot as plt
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

In [121]:
data_orig = np.genfromtxt('data/housepricedata.csv',delimiter=',',skip_header=1)
print("Dataset : \n\n"+ str(data_orig))
print("\nDimensions of dataset : "+str(data_orig.shape))

Dataset : 

[[8.450e+03 7.000e+00 5.000e+00 ... 0.000e+00 5.480e+02 1.000e+00]
 [9.600e+03 6.000e+00 8.000e+00 ... 1.000e+00 4.600e+02 1.000e+00]
 [1.125e+04 7.000e+00 5.000e+00 ... 1.000e+00 6.080e+02 1.000e+00]
 ...
 [9.042e+03 7.000e+00 9.000e+00 ... 2.000e+00 2.520e+02 1.000e+00]
 [9.717e+03 5.000e+00 6.000e+00 ... 0.000e+00 2.400e+02 0.000e+00]
 [9.937e+03 5.000e+00 6.000e+00 ... 0.000e+00 2.760e+02 0.000e+00]]

Dimensions of dataset : (1460, 11)


In [122]:
#Seed for np.random
seed=9
np.random.seed(seed)

In [123]:
#Shuffling imported original dataset
np.random.shuffle(data_orig)

In [124]:
#Shuffled dataset
print("Shuffled dataset with (Seed "+str(seed) +") :\n\n"+str(data_orig))
print("\n"+str(data_orig.shape))

Shuffled dataset with (Seed 9) :

[[3.9640e+03 6.0000e+00 4.0000e+00 ... 1.0000e+00 5.7600e+02 0.0000e+00]
 [3.9104e+04 7.0000e+00 7.0000e+00 ... 2.0000e+00 4.3900e+02 1.0000e+00]
 [6.0400e+03 4.0000e+00 5.0000e+00 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 ...
 [8.7770e+03 5.0000e+00 7.0000e+00 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [2.4480e+03 7.0000e+00 5.0000e+00 ... 0.0000e+00 4.7400e+02 0.0000e+00]
 [9.2450e+03 7.0000e+00 5.0000e+00 ... 0.0000e+00 6.3900e+02 1.0000e+00]]

(1460, 11)


In [125]:
#Extacting Y
y_orig = data_orig[:,-1]
print("Output Y   :"+str(y_orig))
print("Shape of Y : "+str(y_orig.shape))

Output Y   :[0. 1. 0. ... 0. 0. 1.]
Shape of Y : (1460,)


In [126]:
Y = np.reshape(y_orig,(y_orig.shape[0],1)).T    
print("Shape of Y: "+ str(Y.shape))

Shape of Y: (1, 1460)


In [127]:
#Extracting vectorized input feature X (transposed)
x_shuffled = data_orig[:,0:-1].T
print("Input set : \n\n" +str(x_shuffled))

Input set : 

[[3.9640e+03 3.9104e+04 6.0400e+03 ... 8.7770e+03 2.4480e+03 9.2450e+03]
 [6.0000e+00 7.0000e+00 4.0000e+00 ... 5.0000e+00 7.0000e+00 7.0000e+00]
 [4.0000e+00 7.0000e+00 5.0000e+00 ... 7.0000e+00 5.0000e+00 5.0000e+00]
 ...
 [1.0000e+01 5.0000e+00 6.0000e+00 ... 4.0000e+00 6.0000e+00 8.0000e+00]
 [1.0000e+00 2.0000e+00 0.0000e+00 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [5.7600e+02 4.3900e+02 0.0000e+00 ... 0.0000e+00 4.7400e+02 6.3900e+02]]


In [128]:
print(x_shuffled.shape)
X=x_shuffled

(10, 1460)


In [129]:
#Splitting into Train, Test sets ( with a fixed seed )
train_split_percent = 80
test_split_percent = 20

train_X , test_X = X[:, : int( (train_split_percent/100)*X.shape[1])] , X[:,int( (train_split_percent/100)*X.shape[1]) : ]
train_Y , test_Y = Y[:, : int( (train_split_percent/100)*X.shape[1])] , Y[:,int( (train_split_percent/100)*X.shape[1]) : ]
print("Seed of Randomization   : "+str(seed))
print("\nShape of Training set X : "+str(train_X.shape))
print("Shape of Training set Y : "+str(train_Y.shape))
print("\nShape of Test set   X   : "+str(test_X.shape))
print("Shape of Test set Y     : "+str(test_Y.shape))

Seed of Randomization   : 9

Shape of Training set X : (10, 1168)
Shape of Training set Y : (1, 1168)

Shape of Test set   X   : (10, 292)
Shape of Test set Y     : (1, 292)


In [130]:
m_train = train_X.shape[1]
m_test  = test_X.shape[1]
print("No of training examples : "+str(m_train))
print("No of test example      : "+str(m_test))

No of training examples : 1168
No of test example      : 292


In [131]:
def standardize(x):
    """
    Input  :  Numpy array x 
    Output :  Numpy array of same shape as X but standardized along each rows
    
    """
    x_mean = np.mean(x,axis=1, keepdims=True)
    x_std = np.std(x, axis=1, keepdims=True)+0.0000001

    #print("Mean of each row : \n\n"+str(x_mean))
    #print("\nStandard deviation of each row : \n\n"+str(x_std))

    X = (x - x_mean)/x_std   #Python Broadcasting

    return X

In [132]:
train_X = standardize(train_X)
print("Standardize train_X : "+str(train_X.shape)+"\n\n"+str(train_X))
test_X  = standardize(test_X)
print("\n\nStandardize test_X : "+str(test_X.shape)+"\n\n"+str(test_X))

Standardize train_X : (10, 1168)

[[-0.62294439  2.64369886 -0.42995773 ... -0.07112896  0.33101956
  -0.14224398]
 [-0.06530134  0.66109837 -1.51810077 ...  0.66109837  1.38749808
  -0.06530134]
 [-1.39247961  1.28548337 -0.49982528 ... -0.49982528 -0.49982528
  -0.49982528]
 ...
 [ 2.17516164 -0.92614304 -0.30588211 ...  1.5549007   1.5549007
   0.31437883]
 [ 0.59351649  2.15836359 -0.9713306  ...  0.59351649  0.59351649
  -0.9713306 ]
 [ 0.48109632 -0.16278512 -2.22602564 ...  0.85708548  1.9803531
   0.33070066]]


Standardize test_X : (10, 292)

[[-0.09938328  0.03686526 -0.32759958 ... -0.19526819 -1.27316445
  -0.11556279]
 [-1.52084774 -0.09749024 -0.09749024 ... -0.80916899  0.61418851
   0.61418851]
 [ 0.33665213 -0.59072921 -0.59072921 ...  1.26403347 -0.59072921
  -0.59072921]
 ...
 [-1.56607354  0.22958146  0.82813313 ... -1.56607354 -0.36897021
   0.82813313]
 [-0.87582323  0.62853197  0.62853197 ... -0.87582323 -0.87582323
  -0.87582323]
 [-1.08665717  0.03514285 -0.180

In [133]:
def sigmoid(Z):
    return 1/(1+np.exp(-Z))

def relu(Z):
    return np.maximum(0,Z)

def sigmoid_backward(dA, Z):
    sig = sigmoid(Z)
    return dA * sig * (1 - sig)

def relu_backward(dA, Z):
    dZ = np.array(dA, copy = True)
    dZ[Z <= 0] = 0;
    return dZ;

In [134]:
def initialize_parameters(n_x, n_h, n_y):
    
    W1 = np.random.randn(n_h,n_x)*0.1
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(n_y,n_h)*0.1
    b2 = np.zeros((n_y,1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters 

In [135]:
def initialize_parameters_deep(layer_dims):
    parameters = {}
    L = len(layer_dims)            

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l],layer_dims[l-1])*(2/layer_dims[l-1])**0.5
        parameters['b' + str(l)] = np.zeros((layer_dims[l],1))
        
    return parameters

In [136]:
def linear_forward(A, W, b):
   
    Z = np.dot(W,A)+b
    cache = (A, W, b)
    
    return Z, cache

In [137]:
def linear_activation_forward(A_prev, W, b, activation):
    
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev,W,b)
        A, activation_cache = sigmoid(Z), sigmoid(Z)
    
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev,W,b)
        A, activation_cache = relu(Z), relu(Z)
    
    cache = (linear_cache, activation_cache)

    return A, cache

In [138]:
def forwardprop(X, parameters):

    caches = []
    A = X
    L = len(parameters) // 2                
    
    for l in range(1, L):
        A_prev = A 
        A, cache = linear_activation_forward(A_prev, parameters["W"+str(l)], parameters["b"+str(l)], "relu")
        caches.append(cache)
        
    AL, cache = linear_activation_forward(A, parameters["W"+str(L)], parameters["b"+str(L)],activation = "sigmoid")
    caches.append(cache)
            
    return AL, caches

In [139]:
def compute_cost(AL, Y):
    
    m = Y.shape[1]
    cost = (-1/m)*(np.sum(Y*np.log(AL+0.0000001)+(1-Y)*np.log(1.000001-AL)))
    cost = np.squeeze(cost)     
   
    return cost+0.00000001

In [140]:
def linear_backward(dZ, cache):
    
    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = (1/m)*np.dot(dZ,A_prev.T)
    db = (1/m)*np.sum(dZ,axis=1,keepdims=True)
    dA_prev = np.dot(W.T,dZ)

    return dA_prev, dW, db

In [141]:
def linear_activation_backward(dA, cache, activation):

    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = relu_backward(dA,activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA,activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    return dA_prev, dW, db

In [142]:
def backwardprop(AL, Y, caches):
    grads = {}
    L = len(caches) 
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) 
    
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    current_cache = caches[-1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation = "sigmoid")

    
    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 1)], current_cache, activation = "relu")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

In [143]:
def update_parameters(parameters, grads, learning_rate):
    
    L = len(parameters) // 2 

    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l + 1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l + 1)]
        
    return parameters

In [144]:
def initialize_adam(parameters) :

    L = len(parameters) // 2 
    v = {}
    s = {}
    
    for l in range(L):
    
        v["dW" + str(l+1)] = np.zeros((parameters["W" + str(l+1)].shape[0],parameters["W" + str(l+1)].shape[1]))
        v["db" + str(l+1)] = np.zeros((parameters["b" + str(l+1)].shape[0],parameters["b" + str(l+1)].shape[1]))
        s["dW" + str(l+1)] = np.zeros((parameters["W" + str(l+1)].shape[0],parameters["W" + str(l+1)].shape[1]))
        s["db" + str(l+1)] = np.zeros((parameters["b" + str(l+1)].shape[0],parameters["b" + str(l+1)].shape[1]))
    return v, s

In [145]:
def update_parameters_with_adam(parameters, grads, v, s, t, learning_rate = 0.01,
                                beta1 = 0.9, beta2 = 0.999,  epsilon = 1e-8):

    L = len(parameters) // 2                 # number of layers in the neural networks
    v_corrected = {}                         # Initializing first moment estimate, python dictionary
    s_corrected = {}                         # Initializing second moment estimate, python dictionary
    
    # Perform Adam update on all parameters
    for l in range(L):
        v["dW" + str(l+1)] = beta1*v["dW" + str(l+1)]+(1-beta1)*grads['dW'+str(l+1)]
        v["db" + str(l+1)] = beta1*v["db" + str(l+1)]+(1-beta1)*grads['db'+str(l+1)]
       
        v_corrected["dW" + str(l+1)] = v["dW" + str(l+1)]/(1-pow(beta1,t))
        v_corrected["db" + str(l+1)] = v["db" + str(l+1)]/(1-pow(beta1,t))
        
        s["dW" + str(l+1)] = beta2*s["dW" + str(l+1)]+(1-beta2)*np.power(grads['dW'+str(l+1)],2)
        s["db" + str(l+1)] = beta2*s["db" + str(l+1)]+(1-beta2)*np.power(grads['db'+str(l+1)],2)

        s_corrected["dW" + str(l+1)] = s["dW" + str(l+1)]/(1-pow(beta2,t))
        s_corrected["db" + str(l+1)] = s["db" + str(l+1)]/(1-pow(beta2,t))

        parameters["W" + str(l+1)] = parameters["W" + str(l+1)]-learning_rate*np.divide(v_corrected["dW" + str(l+1)],np.sqrt(s_corrected["dW" + str(l+1)])+epsilon)
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)]-learning_rate*np.divide(v_corrected["db" + str(l+1)],np.sqrt(s_corrected["db" + str(l+1)])+epsilon)

    return parameters, v, s

In [146]:
def model(X, Y, layers_dims, learning_rate = 0.003, num_iterations = 3000,
                  beta1 = 0.9, beta2 = 0.999,  epsilon = 1e-8, print_cost=False):

    costs = []                      
    
    parameters = initialize_parameters_deep(layers_dims)
    v, s = initialize_adam(parameters)
    t = 10000
    
    for i in range(0, num_iterations):
        AL, caches = forwardprop(X, parameters)
        cost = compute_cost(AL, Y)
        grads = backwardprop(AL, Y, caches)
        t = t + 0.01
        parameters, v, s = update_parameters_with_adam(parameters, grads, v, s,
                                                               t, learning_rate, beta1, beta2,  epsilon)
        if print_cost and i % 10 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        if print_cost and i % 10 == 0:
            costs.append(cost)
            
    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per thousands)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

In [147]:
def evaluate(Y,Yhat,Set):
    spos=0
    
    for i in range(Y.shape[1]): 
        if Y[0,i]==1 and Yhat[0,i]==1:
            spos = spos+1
            
    p = spos /np.sum(Yhat == 1)
    r = spos/ np.sum( Y == 1)
    acc = np.mean(Y == Yhat)
    f1score = 2*p*r/(p+r)
    
    print(Set+" :       "+str(p) + "  "+str(r)+"  "+str(f1score)+"  "+str(acc))
    
    return 

In [ ]:
parameters = L_layer_model(train_X, train_Y, layers_dims = [10,128,64,32,16,8,4,1], num_iterations =570, 
                           learning_rate = 0.0001,  beta1 = 0.92, beta2 = 0.9,  epsilon = 1e-8, print_cost = True)

def predict(X,parameters):
    
    AL = L_model_forward(X, parameters)[0]
    Y_prediction = AL
    for i in range(AL.shape[1]):
          Y_prediction[0, i] = 1 if AL[0, i] > 0.5 else 0
   
    return Y_prediction 

test_Yhat = predict(test_X,parameters)
train_Yhat = predict(train_X,parameters)


print("    "+" :       "+ "\t Precision " + "  "+ "     \tRecall" +"  "+"          F-score "+"  "+"         Accuracy")

evaluate(train_Y,train_Yhat,"Train")
evaluate(test_Y,test_Yhat,"Test ")



Cost after iteration 0: 0.705895
Cost after iteration 10: 0.693879
Cost after iteration 20: 0.677124
Cost after iteration 30: 0.658920
Cost after iteration 40: 0.640201
Cost after iteration 50: 0.621504
Cost after iteration 60: 0.601854
Cost after iteration 70: 0.581090
Cost after iteration 80: 0.560030
Cost after iteration 90: 0.538762
Cost after iteration 100: 0.517281
Cost after iteration 110: 0.495751
Cost after iteration 120: 0.474306
Cost after iteration 130: 0.452559
Cost after iteration 140: 0.430884
Cost after iteration 150: 0.410029
Cost after iteration 160: 0.389822
Cost after iteration 170: 0.370863
Cost after iteration 180: 0.353254
Cost after iteration 190: 0.337101
Cost after iteration 200: 0.322384
Cost after iteration 210: 0.309213
Cost after iteration 220: 0.297520
Cost after iteration 230: 0.286892
Cost after iteration 240: 0.277575
Cost after iteration 250: 0.269389
Cost after iteration 260: 0.262092
Cost after iteration 270: 0.255786
Cost after iteration 280: 0.250